# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

**Задание 1. Импортируем библиотеку. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`.**

In [1]:
import pandas as pd

try:
    data = pd.read_csv('practicum_datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Задание 2. Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Задание 3. Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Задание 4. Выведем количество пропущенных значений для каждого столбца.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Обнаружены пропуски в столбцах `total_income` и `days_employed`

**Задание 5. Заполним пропуски в столбце `total_income`. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Задание 8. Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**Задание 9. В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Задание 11. Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Проверим пропуски.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Все пропуски заполнены.

### Изменение типов данных

**Задание 13. Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Выведем на экран количество строк-дубликатов в данных.**

In [14]:
data.duplicated().sum()

54

In [15]:
data = data.drop_duplicates()

**Задание 15. Обработаем неявные дубликаты в столбце `education`.**

In [16]:
data['education'] = data['education'].str.lower()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [22]:
data_pivot_1 = data.pivot_table(index= 'total_income_category', columns='debt',values='children', aggfunc='count')
data_pivot_1['mean'] = data_pivot_1.mean(axis = 1)
display(data_pivot_1)

debt,0,1,mean
total_income_category,,,
A,23,2,12.5
B,4660,354,2507.0
C,14585,1353,7969.0
D,328,21,174.5
E,20,2,11.0


## Исследуйте данные и ответьте на вопросы

In [23]:
# Выведем первые 10 строк обработанного датасета:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [24]:
# Создадим сводную таблицу
data_pivot_children = data.pivot_table(index='children', columns='debt', values='total_income_category', aggfunc='count')

# Добавим в сводную таблицу общее количество заемщиков
data_pivot_children['count'] = data_pivot_children[0] + data_pivot_children[1]

# В группе с 5-ю детьми всего 9 человек и все они вернули кредит, поэтому заменим общее число на 9
data_pivot_children.loc[5, ['count']] = 9

# Посчитаем процентное соотношение людей без задолженностей к общему числу по категории колличества детей в семье
data_pivot_children['count_%'] = data_pivot_children['count'] / data_pivot_children['count'].sum() * 100

# Добавим в сводную таблицу процентное отношение людей, вернувших кредит в срок к общему числу 
data_pivot_children['ratio_%'] = data_pivot_children[0] / data_pivot_children['count'] * 100

# Округлим значения и отсортируем по убыванию процента возврата кредита в срок
data_pivot_children.round(3).sort_values(by='ratio_%', ascending=False)

debt,0,1,count,count_%,ratio_%
children,,,,,
5,9.0,NaN,9.0,0.042,100.000
0,13044.0,1063.0,14107.0,66.081,92.465
3,303.0,27.0,330.0,1.546,91.818
1,4365.0,444.0,4809.0,22.527,90.767
2,1858.0,194.0,2052.0,9.612,90.546
4,37.0,4.0,41.0,0.192,90.244


**Задание 19. Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [25]:
# Создадим сводную таблицу
data_pivot_children = data.pivot_table(index='children', columns='debt', values='total_income_category', aggfunc='count')

# Добавим в сводную таблицу общее количество заемщиков
data_pivot_children['count'] = data_pivot_children[0] + data_pivot_children[1]

# В группе с 5-ю детьми всего 9 человек и все они вернули кредит, поэтому заменим общее число на 9
data_pivot_children.loc[5, ['count']] = 9

# Посчитаем процентное соотношение людей без задолженностей к общему числу по категории колличества детей в семье
data_pivot_children['count_%'] = data_pivot_children['count'] / data_pivot_children['count'].sum() * 100

# Добавим в сводную таблицу процентное отношение людей, вернувших кредит в срок к общему числу 
data_pivot_children['ratio_%'] = data_pivot_children[0] / data_pivot_children['count'] * 100

# Округлим значения и отсортируем по убыванию процента возврата кредита в срок
data_pivot_children.round(3).sort_values(by='ratio_%', ascending=False)

debt,0,1,count,count_%,ratio_%
children,,,,,
5,9.0,NaN,9.0,0.042,100.000
0,13044.0,1063.0,14107.0,66.081,92.465
3,303.0,27.0,330.0,1.546,91.818
1,4365.0,444.0,4809.0,22.527,90.767
2,1858.0,194.0,2052.0,9.612,90.546
4,37.0,4.0,41.0,0.192,90.244


**Вывод:**

Стоит обратить внимание на категории с 3-мя, 4-мя и 5-ю детьми.
Количество заемщиков по этим категориям в сумме - **менее** 2% от общего числа заемщиков, поэтому процентное соотношение задолженности по кредиту подвержено большей волатильности и может быстро измениться, относительно категорий заемщиков с 0, 1, 2 детьми.
Отдельно выделим категорию с 5-ю детьми. Процент погашения кредита в срок, объективно, не отражает статистическую действительность, т.к. в данной группе находится всего 9 человек. Даже один человек в данной группе, в перспективе, может кардинально поменять результаты исследования.

*Учитывая вышеизложенные факты, **можно предположить**: риск задолженности по кредиту прямо пропорционален количеству детей в семье.*

**Задание 20. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [26]:
# Создадим сводную таблицу
data_pivot_family = data.pivot_table(index='family_status', columns='debt', values='total_income_category', aggfunc='count')

# Добавим в сводную таблицу общее количество людей по определенному семейному положению
data_pivot_family['count'] = data_pivot_family[0] + data_pivot_family[1]

# Посчитаем процентное соотношение людей без задолженностей к общему числу по определенному семейному положению
data_pivot_family['count_%'] = data_pivot_family['count'] / data_pivot_family['count'].sum() * 100

# Добавим в сводную таблицу процентное отношение людей, вернувших кредит в срок к общему числу 
data_pivot_family['ratio_%'] = data_pivot_family[0] / data_pivot_family['count'] * 100

# Округлим значения и отсортируем по убыванию процента возврата кредита в срок
data_pivot_family.round(3).sort_values(by='ratio_%', ascending=False)

debt,0,1,count,count_%,ratio_%
family_status,,,,,
вдовец / вдова,888,63,951,4.455,93.375
в разводе,1105,84,1189,5.570,92.935
женат / замужем,11339,927,12266,57.457,92.443
гражданский брак,3761,385,4146,19.421,90.714
Не женат / не замужем,2523,273,2796,13.097,90.236


**Вывод:** 

Исходя из данных сводной таблицы, можно констатировать факт - более половины заемщиков состоят в **зарегистрированных** отношениях (категория - "женат / замужем"). Данная категория является медианным значением группы. 

Стоит обратить внимание: т.к. данная категория является медианой, она делит группы на тех, кто, в данный момент, состоит или не состоит в **зарегистрированных** отношениях. Большему риску подвержены группы, **не зарегистрированные** в официальных отношениях, а именно категории: "гражданский брак" и "Не женат / не замужем". Вторая половина - напротив, более ответственно относится к возврату кредита в срок (категории: "в разводе" и "вдовец / вдова"), но стоит акцентировать внимание на том факте, что данные категории составляют приблизительно 10% от всех заемщиков, поэтому, статистические данные подвержены большей волатильности.

*Учитывая вышеизложенные факты, **можно сделать следующие заключение:** по мере приобретения статуса семейного положения - увеличивается ответственность возврата заемных денег в срок.*

**Задание 21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [27]:
# Создадим сводную таблицу
data_pivot_income = data.pivot_table(index='total_income_category', columns='debt', values='gender', aggfunc='count')

# Добавим в сводную таблицу общее количество людей по категории уровня дохода
data_pivot_income['count'] = data_pivot_income[0] + data_pivot_income[1]

# Посчитаем процентное соотношение людей без задолженностей к общему числу по категории уровня дохода
data_pivot_income['count_%'] = data_pivot_income['count'] / data_pivot_income['count'].sum() * 100

# Добавим в сводную таблицу процентное отношение людей, вернувших кредит в срок к общему числу 
data_pivot_income['ratio_%'] = data_pivot_income[0] / data_pivot_income['count'] * 100

# Округлим значения и отсортируем по убыванию процента возврата кредита в срок
data_pivot_income.round(3).sort_values(by='ratio_%', ascending=False)


debt,0,1,count,count_%,ratio_%
total_income_category,,,,,
D,328,21,349,1.635,93.983
B,4660,354,5014,23.487,92.940
A,23,2,25,0.117,92.000
C,14585,1353,15938,74.658,91.511
E,20,2,22,0.103,90.909


**Вывод:** 

Стоит обратить внимание на категории "A", "D" и "E". Количество заемщиков по этим категориям в сумме - менее 2% от общего числа заемщиков, поэтому процентное соотношение задолженности по кредиту подвержено большей волатильности и может быстро поменяется, относительно категорий "В" и "С".

Почти 3/4 всего числа заемщиков - это категория "С" с доходом 50 000 - 200 000 рублей. 

**Учитывая тот факт, что 3 категории из 5 составляют менее 2% от общего числа заемщиков, можно сказать о том, что статистические данные могут не отражать объективные данные. В данном случае не рекомендуется выстраивать взаимосвязь на основе данного анализа.**

**Задание 22. Как разные цели кредита влияют на его возврат в срок?**

In [28]:
# Создадим сводную таблицу
data_pivot_target = data.pivot_table(index='purpose_category', columns='debt', values='total_income_category', aggfunc='count')

# Добавим в сводную таблицу общее количество людей по цели заема денежных средств
data_pivot_target['count'] = data_pivot_target[0] + data_pivot_target[1]

# Посчитаем процентное соотношение людей без задолженностей к общему числу по цели заема денежных средств
data_pivot_target['count_%'] = data_pivot_target['count'] / data_pivot_target['count'].sum() * 100

# Добавим в сводную таблицу процентное отношение людей, вернувших кредит в срок к общему числу 
data_pivot_target['ratio_%'] = data_pivot_target[0] / data_pivot_target['count'] * 100

# Округлим значения и отсортируем по убыванию процента возврата кредита в срок
data_pivot_target.round(2).sort_values(by='ratio_%', ascending=False)


debt,0,1,count,count_%,ratio_%
purpose_category,,,,,
операции с недвижимостью,9974,780,10754,50.37,92.75
проведение свадьбы,2141,183,2324,10.89,92.13
получение образования,3620,369,3989,18.69,90.75
операции с автомобилем,3881,400,4281,20.05,90.66


**Вывод:** 

Стоит отметить, категория "операции с недвижимостью" имеет наименьший риск задолженностей. Данная категория составляет чуть более половины всех заемщиков. Наибольшее количество задолженностей у категории "операции с автомобилем".

В целом в анализе нет явных аномалий и взаимосвязей. *Статистические данные можно считать объективными.*

**Задание 23. Приведите возможные причины появления пропусков в исходных данных.**

*Ответ:* 

1. Неправилная подгрузка/выгрузка данных.
2. Неправильное заполнение данных.
3. Потеря данных при операциях в работе с данными.

**Задание 24. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

*Ответ:* 

Для достоверности исследования (особенно в случае необходимости использования определенных значений), нужно заполнить пропуски или отсутствие значениями для корректного подсчета. 

Медианное значение подходит лучше среднего, т.к. она не учитывает дисперсию исследуемого диапазона. В случае среднего значения, данные будут сильнее искажать желаемый финальный результат.

## Общий вывод.

Семейное положение, наличие детей, уровень дохода и цель кредита влияет на факт погашения кредита в срок.

лица без детей, со средним или выше среднего доходом, берущие кредит с целью улучшения жилищных условий или на другие операции с недвижимостью, являются наиболее надежными заёмщиками;

лица, не состоящие в официальном браке, но имеющие детей, с низким доходом, желающие потратить заёмные средства на покупку автомобиля, наиболее склонны к нарушению договорных обязательств и погашению кредита в срок.

В ходе обзора данных выявили, что:

В таблице 12 столбцов и 21525 строк. Типы данных — object, int, float
Есть пропущенные значения в двух столбцах: days_employed и total_income
Доля пропущенных значений 10,1% - значительная часть данных
Есть отрицательные значения в столбцах children и days_employed
Тип данных в total_income нужно поменять на целочисленный int
В столбце education, есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв
В ходе предобработки данных:

Заполнили пропуски в столбце total_income медианным значением по каждому типу из столбца income_type
Заменили все отрицательные значения в столбце days_employed положительными
Удалили строки с аномальными значениями в стлобце children
Заполнили пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type
Заменили вещественный тип данных в столбце total_income на целочисленный
Обработали неявные дубликаты в столбце education
Данные категоризировали по уровню дохода и по цели взятия кредита